# Best of Each Molecule Type OOF Score

In [67]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [2]:
tracking = pd.read_csv('../tracking/tracking.csv')

In [62]:
tracking.tail(10)

,run_id,model_number,eval_metric,n_estimators,early_stopping_rounds,random_state,n_threads,learning_rate,n_fold,model_type,training_time,n_features,gml_oof_score,oof_score,lb_score,1JHC_mae_cv,2JHH_mae_cv,1JHN_mae_cv,2JHN_mae_cv,2JHC_mae_cv,3JHH_mae_cv,3JHC_mae_cv,3JHN_mae_cv,1JHC_std_mae_cv,2JHH_std_mae_cv,1JHN_std_mae_cv,2JHN_std_mae_cv,2JHC_std_mae_cv,3JHH_std_mae_cv,3JHC_std_mae_cv,3JHN_std_mae_cv,1JHC_tr_sec_f1,1JHC_pred_sec_f1,1JHC_tr_sec_f2,1JHC_pred_sec_f2,2JHH_tr_sec_f1,2JHH_pred_sec_f1,2JHH_tr_sec_f2,2JHH_pred_sec_f2,1JHN_tr_sec_f1,1JHN_pred_sec_f1,1JHN_tr_sec_f2,1JHN_pred_sec_f2,2JHN_tr_sec_f1,2JHN_pred_sec_f1,2JHN_tr_sec_f2,2JHN_pred_sec_f2,2JHC_tr_sec_f1,2JHC_pred_sec_f1,2JHC_tr_sec_f2,2JHC_pred_sec_f2,3JHH_tr_sec_f1,3JHH_pred_sec_f1,3JHH_tr_sec_f2,3JHH_pred_sec_f2,3JHC_tr_sec_f1,3JHC_pred_sec_f1,3JHC_tr_sec_f2,3JHC_pred_sec_f2,3JHN_tr_sec_f1,3JHN_pred_sec_f1,3JHN_tr_sec_f2,3JHN_pred_sec_f2,1JHC_csv_save_sec,2JHH_csv_save_sec,1JHN_csv_save_sec,2JHN_csv_save_sec,2JHC_csv_save_sec,3JHH_csv_save_sec,3JHC_csv_save_sec,3JHN_csv_save_sec
8,0623_1435,M019,group_mae,50000,500,529,52,0.3,2,lgbm,13891.47,44,1.6189,12.0371,-0.664,1.5682,0.3802,0.6467,0.4467,0.7219,0.4151,0.7557,0.2926,0.003040,0.001030,0.005440,0.001810,0.001520,0.000240,0.001420,0.000510,346.10,100.98,368.60,105.67,148.53,30.95,241.49,52.45,58.65,2.66,29.91,1.00,109.09,9.83,128.56,11.58,1255.46,606.45,1287.23,751.41,525.90,228.06,613.84,366.80,1617.83,1298.30,1570.21,1370.32,170.59,26.13,147.59,20.96,24.21,24.53,24.55,24.57,25.37,25.68,26.69,NaN
9,0624_0121,M020,group_mae,500000,500,529,60,0.3,2,lgbm,21487.80,44,1.6188,12.0366,-0.666,1.5631,0.3787,0.6449,0.4429,0.7222,0.4145,0.7547,0.2919,0.005260,0.000770,0.004840,0.001810,0.002260,0.001040,0.000900,0.001920,810.07,209.61,699.04,129.85,236.61,40.68,369.15,63.30,25.17,0.54,38.90,1.34,104.71,8.09,134.67,10.52,1440.64,563.56,1718.44,832.42,943.03,390.52,1002.29,508.39,2007.33,1489.61,3726.66,3186.55,218.05,27.95,235.06,28.37,23.97,23.97,23.98,24.05,25.12,25.29,26.24,NaN
10,0626_0129,M021,group_mae,500000,500,529,60,0.3,2,lgbm,21636.09,41,1.5972,11.9581,-1.040,1.1473,0.3138,0.5158,0.1901,0.4974,0.2637,0.5558,0.2039,0.001680,0.000490,0.004510,0.000680,0.000760,0.000290,0.002460,0.000850,921.21,246.51,573.02,120.05,295.11,49.60,243.07,43.06,52.96,1.76,84.49,2.54,146.22,9.50,195.90,13.06,2766.36,896.12,3656.52,2036.66,707.07,387.69,803.66,414.42,1778.43,1304.23,1761.38,1199.73,355.86,30.79,266.45,23.42,24.82,25.27,25.63,25.41,25.80,26.56,27.70,NaN
11,0627_0017,M023,group_mae,500000,500,529,48,0.3,2,lgbm,44349.00,167,-1.1118,0.4420,-1.264,0.8204,0.2090,0.4658,0.1782,0.3776,0.2506,0.4922,0.2068,0.002274,0.000401,0.001344,0.002496,0.001249,0.001496,0.000714,0.000173,1453.00,998.00,1351.00,1080.00,724.00,336.00,748.00,341.00,56.00,4.00,56.00,4.00,192.00,27.00,181.00,26.00,3510.00,2903.00,3379.00,2974.00,1484.00,1113.00,920.00,699.00,3731.00,3967.00,5494.00,5735.00,233.00,65.00,229.00,46.00,25.00,26.00,26.00,26.00,27.00,28.00,30.00,NaN
12,0627_1434,M024,group_mae,500000,500,529,48,0.1,2,lgbm,147803.00,99,-1.2826,0.3771,-1.382,0.7073,0.1739,0.3909,0.1478,0.3198,0.2082,0.4221,0.1750,0.000606,0.000376,0.001168,0.001002,0.000282,0.000295,0.000301,0.001704,3888.00,4425.00,3707.00,4235.00,1976.00,1339.00,2079.00,1456.00,126.00,10.00,181.00,18.00,497.00,159.00,539.00,171.00,11349.00,14801.00,11535.00,14338.00,3638.00,3812.00,2954.00,3374.00,9699.00,14332.00,12831.00,17854.00,870.00,364.00,648.00,279.00,26.00,26.00,26.00,26.00,28.00,29.00,30.00,NaN
13,0629_1311,M025,group_mae,500000,500,529,24,0.1,2,catboost,30069.00,99,-1.3265,0.3533,-1.415,0.6691,0.1640,0.4348,0.1398,0.3002,0.1934,0.3905,0.1627,0.000387,0.000594,0.022253,0.001201,0.000589,0.000427,0.000546,0.001450,2028.00,65.00,1623.00,52.00,1130.00,17.00,1029.00,27.00,829.00,0.00,419.00,0.00,421.00,3.00,349.00,2.00,3297.00,160.00,3414.00,160.00,2297.00,170.00,2448.00,72.00,4219.00,218.00,4075.00,156.00,512.00,7.00,556.00,7.00,25.00,25.00,26.00,25.00,30.00,33.00,37.00,NaN


In [9]:
cvs = ['1JHC_mae_cv', '2JHH_mae_cv', '1JHN_mae_cv',
       '2JHN_mae_cv', '2JHC_mae_cv', '3JHH_mae_cv', '3JHC_mae_cv',
       '3JHN_mae_cv']

In [61]:
sub = pd.read_csv('../input/test.csv')
oof = pd.read_csv('../oof/M023_0627_0017_oof_lgb_2folds_-1.1118CV_500000iter_0.3lr.csv')
oof = oof.drop('oof_preds', axis=1)
oof['oof'] = 0

for cv in tqdm(cvs):
    run_id = tracking.loc[tracking[cv] == tracking.drop([15, 16, 17])[cv].min()]['run_id'].values[0]
    print(f'For {cv} best run_id is {run_id}')
    sub_file = [x for x in os.listdir('../submissions/') if run_id in x][0]
    sub_temp = pd.read_csv('../submissions/{}'.format(sub_file))

    oof_file = [x for x in os.listdir('../oof/') if run_id in x][0]
    oof_temp = pd.read_csv('../oof/{}'.format(oof_file))

    sub.loc[sub['type'] == cv[:4], 'scalar_coupling_constant'] = sub_temp.loc[sub['type'] == cv[:4]]['scalar_coupling_constant'].values
    oof.loc[oof['type'] == cv[:4], 'oof'] = oof_temp.loc[oof_temp['type'] == cv[:4]]['oof_preds'].values

  0%|          | 0/8 [00:00<?, ?it/s]

For 1JHC_mae_cv best run_id is 0629_1311


 12%|█▎        | 1/8 [00:05<00:35,  5.00s/it]

For 2JHH_mae_cv best run_id is 0629_1311


 25%|██▌       | 2/8 [00:10<00:30,  5.08s/it]

...

 75%|███████▌  | 6/8 [00:30<00:09,  4.99s/it]

For 3JHC_mae_cv best run_id is 0629_1311


 88%|████████▊ | 7/8 [00:35<00:04,  4.97s/it]

For 3JHN_mae_cv best run_id is 0629_1311


100%|██████████| 8/8 [00:41<00:00,  5.12s/it]


In [64]:
from sklearn.metrics import mean_absolute_error
def group_mean_log_mae(y_true, y_pred, groups, floor=1e-9):
    maes = (y_true-y_pred).abs().groupby(groups).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

In [69]:
group_mean_log_mae(oof['scalar_coupling_constant'], oof['oof'], oof['type'])

-1.339827955914136